In [19]:
# %%
from __future__ import print_function
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import math
from sklearn.metrics import confusion_matrix
from ipywidgets import interact, interactive, fixed, interact_manual,Layout
import ipywidgets as widgets
from matplotlib import pyplot
import matplotlib.pyplot as plt


ServerName = ""
Database = ""
UserPwd = ""
Driver = ""
engine = create_engine('mssql+pyodbc://' + UserPwd + '@' + ServerName + '/' + Database + "?" + Driver)

# gets a list of paitent csn id to get for the demo
def get_pat_csn_list():

    pat_csn_list=[]

    get_pat_csn="""select top 10 [pat_enc_csn_id] from (
                        SELECT DISTINCT [pat_enc_csn_id] FROM <Covid feature z scored table see detail in spec sheet>         
                   ) x
                   order by  NEWID()"""
    
    get_pat_csn_results=pd.read_sql(get_pat_csn, engine)
    print(get_pat_csn_results.sample())
    
    for row in get_pat_csn_results.itertuples():
        #adds the pat_enc_csn_id to the list as a string so it can be selected from a dropdown list
        pat_csn_list.append(str(int(row[1])))
    return pat_csn_list

    
    
def retriveData (rnn=12,patients=2,hours_ahead=0,n_meds=1,n_covs=1,enc_csn_id='1'):
    #arrays to be populated and returned
    num_obs_time=[]
    num_obs_values=[]
    num_rnn_grid_times=[]
    rnn_grid_times=[]
    labels=[]
    T=[]
    Y=[]
    ind_kf=[]
    ind_kt=[]
    meds_on_grid=[]
    baseline_covs=[]

    
    print('Starting retrive  data...')
    # Create the connection
    engine = create_engine('mssql+pyodbc://' + UserPwd + '@' + ServerName + '/' + Database + "?" + Driver)
    # first selects the distinct encounters
    if patients == 0:
        select_pats=f""" SELECT DISTINCT [pat_enc_csn_id] FROM <Covid feature z scored table see detail in spec sheet>"""
    if enc_csn_id != '1':
        select_pats=f"""SELECT [pat_enc_csn_id] 
                        FROM <Covid feature z scored table see detail in spec sheet> 
                        WHERE pat_enc_csn_id={enc_csn_id}"""
    else:
        select_pats=f""" SELECT DISTINCT TOP {patients}[pat_enc_csn_id] FROM <Covid feature z scored table see detail in spec sheet>"""
    
    pat_enc=pd.read_sql(select_pats, engine)
    baseline_covs= np.zeros((len(pat_enc),n_covs))
    
    
    i=0
    for row in pat_enc.itertuples():
        
        pats_enc_id=row[1]

        #then for each encounter get a list of all data points taken
        select_enc_data=f"""select * from (
                                SELECT z_score_ord_num_value, component_name,hours_since_admitted,
                                b.discharge_disposition,
                                ind_kt
                                ,lengh_of_stay
                                FROM <Covid feature z scored table see detail in spec sheet> a 
                                left join <discharge disposition table see detail in spec sheet > b on a.pat_enc_csn_id=b.pat_enc_csn_id
                                WHERE 
                                a.[pat_enc_csn_id]='{pats_enc_id}'
                            )
                            where [hours_since_admitted] < lengh_of_stay - {hours_ahead}
                            ORDER BY [hours_since_admitted]"""

        
        pat_enc=pd.read_sql(select_enc_data,engine)
        # and tranforms them into the proper format
        pat_y,pat_ind_kf,pat_ind_kt,pat_label,pat_T=transformData(pat_enc)
        #before appending them to the relavent mesures
        pat_T=list(sorted(set(pat_T),key=lambda x: pat_T.index(x)))
        Y.append(np.array(pat_y))
        ind_kf.append(np.array(pat_ind_kf).astype(int))
        ind_kt.append(np.array(pat_ind_kt).astype(int))
        #then goes on to calcualted or retried other info needed
        
        #the T values for just this paitent
        T.append(np.array(pat_T))
        #number of times this paitent was observed
        num_obs_time.append(len(pat_T))
        #and how many mesurements were developed
        num_obs_values.append(len(pat_y))
        #calculates end time
        end_time=max(pat_T)
        #calculets the num_rnn_gird times observed by dividing the end time by 12 and rounding down

        num_rnn_grid_times.append(math.floor(end_time/rnn)+1)
        #rnn_grid_times.append(np.arange(num_rnn_grid_times[i]))
        rnn_grid_times.append((np.arange(start=0,stop=np.floor(end_time)+1,step=rnn,dtype="int")))
        #adds label to grid
        labels.append(int(pat_label))
        #files in the med grid with zeros
        #meds_on_grid.append(np.zeros((num_rnn_grid_times[i],n_meds),dtype="int"))
            
        meds = np.array(np.zeros((num_rnn_grid_times[i],n_meds),dtype="int"))
        meds_on_grid.append(meds)
        
        #baseline_covs.append(np.zeros(n_covs,dtype="int"))

        #grabs the label for the 

        i+=1
        #converts all arrays to numpy arrays before returning them
    
    num_obs_time=np.array(num_obs_time)
    num_obs_values=np.array(num_obs_values)
    num_rnn_grid_times=np.array(num_rnn_grid_times)
    rnn_grid_times=np.array(rnn_grid_times)
    labels=np.array(labels)
    T=np.array(T)
    Y=np.array(Y)
    ind_kf=np.array(ind_kf)
    ind_kt=np.array(ind_kt)
    meds_on_grid=np.array(meds_on_grid)
    baseline_covs=np.array(baseline_covs)
    
    return num_obs_time,num_obs_values,num_rnn_grid_times,rnn_grid_times,labels,T,Y,ind_kf,ind_kt,meds_on_grid,baseline_covs

#takes the dataframe from the database and tranforms its into the correct 2 dimensional array
def transformData (patDataFrame):
    
    #the array the holds the value of the measurements
    y=[]
    #array that holds what value the measurements was
    ind_kf=[]
    #array that holds the time the measurements was taken at
    ind_kt=[]
    T=[]
    pat_label=0

    features = [  "RESPIRATIONS", "TEMPERATURE", "MAP", "PULSE",  "PULSE OXIMETRY", "BMI", "R FIO2",
            "R OXYGEN FLOW RATE", "R OXYGEN DEVICE",
             "C-REACTIVE PROTEIN", "FERRITIN",  "ABSOLUTE LYMPH CT",
             "R LH SEPSIS WATCH SCORE"]
    feature_indices = list(range(len(features)))
    # We need to index the features by 
    measurementNameKF =dict(zip(features,feature_indices))
    #itterates through the dataframe passed in to populate the array
    i=0
    for row in patDataFrame.itertuples():
        if i == 0:
            pat_label=row[4]
        y.append(row[1])
        #converts the measurement name to a number before adding it
        ind_kf.append(measurementNameKF[row[2]])
        ind_kt.append(row[5])
        T.append(row[3])
    return y,ind_kf,ind_kt,pat_label,T

def pad_rawdata(T,Y,ind_kf,ind_kt,X,meds_on_grid,covs):

    N = np.shape(T)[0] #num in batch
    num_meds = np.shape(meds_on_grid[0])[1]
    num_covs = np.shape(covs)[1]
  
    
    T_lens = np.array([len(t) for t in T])
    T_maxlen = np.max(T_lens)
    T_pad = np.zeros((N,T_maxlen))
    
    Y_lens = np.array([len(y) for y in Y])
    Y_maxlen = np.max(Y_lens) 
    Y_pad = np.zeros((N,Y_maxlen))
    ind_kf_pad = np.zeros((N,Y_maxlen))
    ind_kt_pad = np.zeros((N,Y_maxlen))
    grid_lens = np.array([np.shape(m)[0] for m in meds_on_grid])
    grid_maxlen = np.max(grid_lens)
    meds_cov_pad = np.zeros((N,grid_maxlen,num_meds+num_covs))
    X_pad = np.zeros((N,grid_maxlen))
    
    for i in range(N):
       

        T_pad[i,:T_lens[i]] = T[i]
        Y_pad[i,:Y_lens[i]] = Y[i]
        ind_kf_pad[i,:Y_lens[i]] = ind_kf[i]
        ind_kt_pad[i,:Y_lens[i]] = ind_kt[i]
        X_pad[i,:grid_lens[i]] = X[i]
        meds_cov_pad[i,:grid_lens[i],:num_meds] = meds_on_grid[i]
        meds_cov_pad[i,:grid_lens[i],num_meds:] = np.tile(covs[i],(grid_lens[i],1))

                    
    return T_pad,Y_pad,ind_kf_pad,ind_kt_pad,X_pad,meds_cov_pad

# gets and prints the patents discharge disposition
def get_discharge_disposition(enc_csn_id):
    get_discharge_disposition= """SELECT discharge_disposition
                                    FROM <discharge disposition table see detail in spec sheet>
                                        WHERE PAT_ENC_CSN_ID=?"""
    get_discharge_disposition_res=pd.read_sql(get_discharge_disposition,engine,params=[enc_csn_id])

    for row in get_discharge_disposition_res.itertuples():
        print("Discharge Disposition = "+str(int(row[1]))+ " [Training Label]")
        
def get_discharge_disposition_epic(enc_csn_id):
    get_discharge_disposition= """SELECT name
                                    FROM <discharge disposition table see detail in spec sheet>
                                        WHERE PAT_ENC_CSN_ID=?"""
    get_discharge_disposition_res=pd.read_sql(get_discharge_disposition,engine,params=[enc_csn_id])

    for row in get_discharge_disposition_res.itertuples():
        print("Discharge Disposition Status from Epic was '"+str(row[1]) + "'")
        
    
def get_graphs2(enc_csn_id):
    #list to hold which figures to have values to add
    measurements=[]
    #dict that holds the label for the y axis of the possible graphs
    y_axis_label= {"R LH SEPSIS WATCH SCORE":"R LH SEPSIS WATCH SCORE",
                  "FERRITIN":"FERRITIN",
                  "C-REACTIVE PROTEIN":"C-REACTIVE PROTEIN",
                  "ABSOLUTE LYMPH CT":"ABSOLUTE LYMPH CT",
                  "RESPIRATIONS":"RESPIRATIONS",
                  "PULSE":"PULSE",
                  "PULSE OXIMETRY":"PULSE OXIMETRY",
                  "TEMPERATURE":"TEMPERATURE",
                  "R FIO2":"R FIO2",
                  "R OXYGEN FLOW RATE":"R OXYGEN FLOW RATE",
                  "MAP":"MAP",
                  "BMI":"BMI",
                  "age":"age"}
    #dict that holdss the dataframe returned
    dataframes={}
    
    features=["R LH SEPSIS WATCH SCORE",
             "FERRITIN",
             "C-REACTIVE PROTEIN",
             "ABSOLUTE LYMPH CT",
             "RESPIRATIONS",
             "PULSE",
             "PULSE OXIMETRY",
             "TEMPERATURE",
             "R FIO2",
             "R OXYGEN FLOW RATE",
             "MAP",
             "BMI",
              "age"]
    
    
    for s_feature in features:
        q_select = f""" select hours_since_admitted  , ord_num_value '{s_feature}'  from <covid_feature table see detail in spec sheet>
                            where pat_enc_csn_id='{enc_csn_id}'
                            and component_name='{s_feature}'
                            order by hours_since_admitted"""
        q_data=pd.read_sql(q_select, engine)
        if not q_data.empty:
            measurements.append(s_feature)
            dataframes.update({s_feature:q_data})

    

    i = 1
    for measure in measurements:
        fig=plt.figure()
        fig.set_size_inches(18.5, 3)
        axis=plt.subplot()
        if  (dataframes[measure].size ) > 3:
            dataframes[measure].plot.line(x="hours_since_admitted",y=y_axis_label[measure],ax=axis)
        else:
            dataframes[measure].plot.scatter(x="hours_since_admitted",y=y_axis_label[measure],ax=axis)
        i= i + 1
        plt.savefig("/tf/plt_output/"+measure+"_output.png") 

    plt.show()





def get_prediction(n_hours_ahead=24,s_enc_csn_id='218348727'):
    seed = 8675309
    rs = np.random.RandomState(seed) #fixed seed in np
    
    #####
    ##### Setup ground truth and sim some data from a GP
    #####
    num_encs= 1
    M=20
    n_covs=1
    n_meds=1
    Ntr = 1
    batch_size = 1
    tf.compat.v1.disable_eager_execution()
    
    (num_obs_times,num_obs_values,num_rnn_grid_times,rnn_grid_times,labels,times,
       values,ind_lvs,ind_times,meds_on_grid,covs) = retriveData(6,num_encs,n_hours_ahead,enc_csn_id=s_enc_csn_id)
    
    print ("pats_enc_id = " + s_enc_csn_id)
    get_graphs2(s_enc_csn_id)
       
    T_pad_te,Y_pad_te,ind_kf_pad_te,ind_kt_pad_te,X_pad_te,meds_cov_pad_te = pad_rawdata(
                    times,values,ind_lvs,ind_times,rnn_grid_times,meds_on_grid,covs) 
    
    num_obs_times_te = num_obs_times
    num_obs_values_te = num_obs_values
    num_rnn_grid_times_te = num_rnn_grid_times
    labels_te = labels
    
    sess = tf.compat.v1.Session()
    saver = tf.compat.v1.train.import_meta_graph("./EWS_output/EWS_model.meta")
    saver.restore(sess,tf.train.latest_checkpoint('./EWS_output/'))
    
    preds= sess.graph.get_tensor_by_name('preds:0')
    probs= sess.graph.get_tensor_by_name('probs:0')
    Y = sess.graph.get_tensor_by_name('Y:0')
    T = sess.graph.get_tensor_by_name('T:0')
    ind_kf = sess.graph.get_tensor_by_name('ind_kf:0')
    ind_kt = sess.graph.get_tensor_by_name('ind_kt:0')
    X = sess.graph.get_tensor_by_name('X:0')
    med_cov_grid = sess.graph.get_tensor_by_name('med_cov_grid:0')
    O = sess.graph.get_tensor_by_name('O:0')
    num_obs_times = sess.graph.get_tensor_by_name('num_obs_times:0')
    num_obs_values = sess.graph.get_tensor_by_name('num_obs_values:0')
    num_rnn_grid_times = sess.graph.get_tensor_by_name('num_rnn_grid_times:0')
    
    feed_dict={Y:Y_pad_te,T:T_pad_te,ind_kf:ind_kf_pad_te,ind_kt:ind_kt_pad_te,X:X_pad_te,
                       med_cov_grid:meds_cov_pad_te,num_obs_times:num_obs_times_te,
                       num_obs_values:num_obs_values_te,num_rnn_grid_times:num_rnn_grid_times_te}
    #,O:labels_te}    
    
    #print (labels_te)
    te_probs  = sess.run([probs],feed_dict)    

    print("Prediction = " + str( te_probs[0]))
    get_discharge_disposition(s_enc_csn_id)
    get_discharge_disposition_epic(s_enc_csn_id)
    get_graphs2(s_enc_csn_id)


if __name__ == "__main__":
    pats=get_pat_csn_list()
    style = {'description_width': 'initial'}
    #creates widgets
    hours_ahead_widg=widgets.IntSlider(description="Hours ahead to predict",value=24,min=12,max=48,step=1,style=style,layout=Layout(width='350px'))
    pat_widg=widgets.Dropdown(options=pats,description="Random 10 Encounter ID",style=style,layout=Layout(width='250px'))

    interact_manual(
                        get_prediction,
                        n_hours_ahead=hours_ahead_widg,
                        s_enc_csn_id=pat_widg
                    )

# %%


   pat_enc_csn_id
3     218826341.0


interactive(children=(IntSlider(value=24, description='Hours ahead to predict', layout=Layout(width='350px'), …